# The (extended) pairing Hamiltonian

## This version has been solved up to Problem 5. Please skip to that part of the notebook, executing all the code before it without modifying anything.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt


The pairing Hamiltonian is a schematic, but powerful Hamiltonian used to explore the emergence of superfluidity. It is a standard benchmark problem for quantum many-body approaches as it has been exactly solved by Richardson [[Richardson, Phys. Lett. 3, 227 (1963)](https://doi.org/10.1016/0031-9163(63)90259-2)] and poses significant challenges for many methods due to a phase transition to a superfluid phase for sufficiently attractive pairing interaction. It is also simple enough to be an instructive way to learn and explore quantum many-body methods.

The pairing Hamiltonian describes spin-1/2 fermions occupying discrete evenly-space levels. The Hamiltonian to describe this is:

$$
H_1 = \Delta ϵ \sum_{p} \sum_{\sigma} (p - 1) a^{\dagger}_{p\sigma} a_{p\sigma}.
$$

$\Delta ϵ > 0$ is the spacing between levels. $p$ is the quantum number for the levels from $p = 1$, ..., $p = p_\mathrm{max}$. $\sigma$ indicates whether a fermion is spin up ($\sigma = +1$) or spin down ($\sigma = -1$).

### Problem 1: Single-particle basis

Write a function to build a list of all possible states $|p \sigma\rangle$ a single fermion can be for a given $p_\mathrm{max}$.

In [ ]:
def make_list_of_single_particle_states(pmax):
    '''
    Creates a basis of single-particle states for the pairing Hamiltonian given a number of levels pmax.

    Args:
        pmax: integer >= 1 indicating the number of levels

    Returns:
        List of states (p, sigma) in the single-particle basis.
    '''
    
    states = []

    # My solution
    for p in range(1, pmax + 1):
        for sigma in [-1, 1]:
            states.append((p, sigma))

    return states

This is your single-particle basis. All remaining operators and states can be constructed from this single-particle basis.

> Note: The order of states in your basis is arbitrary. The result of your calculations will not matter how you order your basis *as long as you do everything consistently*. However, you can make life easier for yourself by choosing a well motivated ordering (for example, sorting states from lowest to highest energy).

### Problem 2: 1-body Hamiltonian

Write a function to construct the matrix elements of the 1-body Hamiltonian above, $H_1$:

$$
H_{ij} = \langle i = (p \sigma) | H_1 | j = (p' \sigma') \rangle .
$$

We use the useful abbreviated notation $i = (p \sigma)$ so that we can talk about a generic state $|i\rangle$ without worrying about its quantum numbers.

In [ ]:
def make_1body_hamiltonian(basis, delta_eps):
    '''
    Builds 1-body Hamiltonian for pairing Hamiltonian given a basis and an input coupling delta_eps.

    Args:
        basis: List of single-particle states (p, sigma).
        delta_eps: The one-body coupling for the pairing Hamiltonian.

    Returns:
        1-body Hamiltonian as matrix.
    '''
    h1 = np.zeros((len(basis), len(basis)))

    # My solution
    dim = len(basis)
    for i in range(dim):
        for j in range(dim):
            # (i == j) -> (p_i == p_j) and (sigma_i == sigma_j)
            if i == j:
                p, _ = basis[i]
                h1[i,j] = delta_eps * (p - 1)
    
    return h1

It is generally useful to check that certain symmetry properties of your Hamiltonian are fulfilled. In this case, we can check Hermiticity.

Write a function to check that your 1-body operator is Hermitian.

### Problem 3: Computing state energies

Within a given single-particle basis, one can easily represent a Slater determinant state

$$
|\Phi\rangle = a^\dagger_{i_1} \dots a^\dagger_{i_A} |0\rangle
$$

using the occupation numbers $n_i$ for the states $| i \rangle$:

$$
n_i = \begin{cases}
1 & \mathrm{if}\:i\in {i_1, \dots, i_A}, \\
0 & \mathrm{otherwise}.
\end{cases}
$$

Write a function to generate occupation numbers from a list of occupied states [($p_1$, $\sigma_1$), ..., ($p_A$, $\sigma_A$)].

In [ ]:
def create_state_occupation_numbers(basis, occupied_states):
    '''
    Generates list of occupation numbers for basis based on given occupied states.

    Args:
        basis: List of single-particle states (p, sigma).
        occupied_states: List of occupied single-particle states (p, sigma).

    Returns:
        occupation_numbers: List of integers, where occupation_numbers[i] = 1 if basis[i] is in occupied_states.
    '''
    assert len(occupied_states) <= len(basis)
    occupation_numbers = [0] * len(basis)
    
    # My solution
    for i, state in enumerate(basis):
        if state in occupied_states:
            occupation_numbers[i] = 1
    
    return occupation_numbers

### Problem 4: Two-body interactions

So far our 1-body Hamiltonian has only provided us a set of discrete levels, but no actual interactions to explore interesting physics. For this reason, the lowest energy state is easy to construct and solves the problem exactly. Adding two-body interactions allows us to bring in nontrivial physics, which will modify the exact ground-state of the system.

The two-body interaction for the pairing Hamiltonian is

$$
\begin{aligned}
H_2 &= -g \sum_{p \sigma} \sum_{q \sigma'} a^\dagger_{p \sigma} a^\dagger_{p \bar{\sigma}} a_{q \bar{\sigma}'} a_{q \sigma'} \\
&= \frac{1}{4} \sum_{ijkl} H_{ijkl} a^\dagger_{i} a^\dagger_{j} a_{l} a_{k}
\end{aligned}
$$
where $\bar{\sigma} = -\sigma$ (the opposite spin). This interaction is between pairs in the same level, which gives this Hamiltonian its name, the "pairing" Hamiltonian.

A function that generates the two-body Hamiltonian matrix elements
$$
H_{ijkl} = \langle i = (p_i \sigma_i), j = (p_j \sigma_j) | H_2 | k = (p_k \sigma_k), l = (p_l \sigma_l) \rangle
$$
for this Hamiltonian is provided below.

In [ ]:
def make_2body_hamiltonian_pairing_interaction(basis, g):
    '''
    Builds 2-body Hamiltonian matrix elements for a given basis and coupling g.

    Args:
        basis: List of single-particle states (p, sigma).
        g: 2-body pairing coupling (g > 0 is attractive).

    Returns:
        h2: 2-body Hamiltonian matrix elements H_{ijkl} as a 4-dimensional array.
    '''
    h2 = np.zeros((len(basis), len(basis), len(basis), len(basis)))

    for i, state_i in enumerate(basis):
        p_i, sigma_i = state_i
        for j, state_j in enumerate(basis):
            p_j, sigma_j = state_j

            # Can skip because of antisymmetry
            if i == j:
                continue
            for k, state_k in enumerate(basis):
                p_k, sigma_k = state_k
                for l, state_l in enumerate(basis):
                    p_l, sigma_l = state_l

                    if k == l:
                        continue

                    # Check that we have pairs in (i, j) and (k, l)
                    if (
                        p_i == p_j
                        and sigma_i != sigma_j
                        and p_k == p_l
                        and sigma_k != sigma_l
                    ):
                        # This logic is needed for antisymmetry
                        if sigma_i == sigma_k:
                            h2[i, j, k, l] = -1 * g
                        else:
                            h2[i, j, k, l] = 1 * g                    

    return h2

It is very useful to check the symmetries of the Hamiltonian, Hermiticity and antisymmetry:

$$
\begin{aligned}
H_{ijkl} &= H_{klij}, \\
H_{ijkl} &= -H_{jikl} = -H_{ijlk} = H_{jilk}.
\end{aligned}
$$

Write functions to check these properties.

In [ ]:
def eval_2body_hamiltonian_nonpairing_interaction_matrix_element(basis, i, j, k, l, g_hop):
    '''
    Evaluates single matrix element H'_{ijkl}.
    '''
    pi, sigi = basis[i]
    pj, sigj = basis[j]
    pk, sigk = basis[k]
    pl, sigl = basis[l]

    if abs(pi - pj) != 1:
        return 0.0
    if abs(pk - pl) != 1:
        return 0.0
    if sigi != sigj:
        return 0.0
    if sigk != sigl:
        return 0.0

    if sigi == sigk:
        return g_hop
    else:
        return -1 * g_hop


def make_2body_hamiltonian_nonpairing_interaction(basis, g_hop):
    '''
    Builds 2-body Hamiltonian matrix elements for the added interaction for a given basis and coupling g_hop.

    Args:
        basis: List of single-particle states (p, sigma).
        g_hop: 2-body nearest neighbor coupling.

    Returns:
        h2: 2-body Hamiltonian matrix elements H_{ijkl} as a 4-dimensional array.
    '''
    dim = len(basis)
    h2 = np.zeros((dim, dim, dim, dim))

    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                for l in range(dim):
                    h2[i, j, k, l] = eval_2body_hamiltonian_nonpairing_interaction_matrix_element(basis, i, j, k, l, g_hop)

    return h2

### Problem 5: Solving the Hartree-Fock equations

The following function will provide you with the 1- and 2-body Hamiltonian matrix elements and the occupations for our problem as a function of the number of levels $p_\mathrm{max}$, the couplings $\Delta \epsilon$ (1-body Hamiltonian), $g$ (2-body pairing term), $g_\mathrm{hop}$ (2-body nonpairing term), and the number of occupied levels $N_\mathrm{occ}$ (total number of fermions is $2 N_\mathrm{occ}$). Use it to set up the following problem. A reasonable set of defaults is:

- $p_\mathrm{max} = 4$,
- $\Delta \epsilon = 1$,
- $g = 2$,
- $g_\mathrm{hop} = -0.5$,
- $N_\mathrm{occ} = 2$.

In [ ]:
def set_up_problem(pmax, delta_eps, g, g_hop, n_occ):
    '''
    Constructs everything necessary to solve HF for the schematic Hamiltonian.

    Args:
        pmax: Number of levels in the system.
        delta_eps: 1-body coupling defining the level spacing.
        g: 2-body coupling for the pairing interaction.
        g_hop: 2-body coupling for the nearest neighbor interaction.
        n_occ: Number of occupied levels.

    Returns:
        h1: 1-body Hamiltonian matrix elements (dimension = dim x dim).
        h2: 2-body Hamiltonian matrix elements with both interactions (dimension = dim x dim x dim x dim).
        occs: Array of occupation numbers (dimension = dim).
    '''
    assert n_occ <= pmax
    occupied = [(p, sigma) for sigma in [-1, 1] for p in range(1, n_occ + 1)]
    basis = make_list_of_single_particle_states(pmax)
    h1 = make_1body_hamiltonian(basis, delta_eps)
    h2 = make_2body_hamiltonian_pairing_interaction(basis, g)
    h2p = make_2body_hamiltonian_nonpairing_interaction(basis, g_hop)
    occs = create_state_occupation_numbers(basis, occupied)

    return h1, h2 + h2p, occs

In [ ]:
# Example usage:
h1, h2, occupation_numbers = set_up_problem(4, 1.0, 2.0, -0.5, 2)

We will now work on solving the Hartree-Fock (HF) equations for this problem. The central object we are interested in is the Fock matrix:

$$
F_{ij} = H_{ij} + \sum_{kl} \rho_{kl} H_{ikjl}.
$$

As you can see, this requires the 1-body density matrix for the new basis $| \bar{i} \rangle = \sum_{i} C_{\bar{i}i} | i \rangle$. Our starting point will always be $| \bar{i} \rangle = | i \rangle$. Still, we need to construct the 1-body density matrix, which we can do according to
$$
\rho_{ij} = \sum_{\bar{i}} C_{\bar{i}i} n_{\bar{i}} C_{\bar{i}j}^{*},
$$
where $n_\bar{i} = n_i$.

Write a function to compute the 1-body density matrix given a set of coefficients $C_{\bar{i}i}$ and occupation numbers $n_\bar{i}$.

In [ ]:
def construct_new_density(coeffs, occupation_numbers):
    '''
    Constructs 1-body density matrix.

    Args:
        coeffs: Matrix of coefficients from the starting basis to the new basis.
        occupation_numbers: List of occupation numbers for single-particle states in the new basis.
        
    Returns:
        density: 1-body density matrix representing the many-body state in the starting basis.
    '''
    dim = len(occupation_numbers)
    assert coeffs.shape == (dim, dim)
    density = np.zeros((dim, dim))

    # TODO: Your code here

    return density

In [ ]:
# Code to check density matrix here.

Now write a function to compute the Fock matrix $F$ based on your Hamiltonian (with 1- and 2-body parts) and a density. For now this density is just the density of the lowest-energy state you found above. For this state, $C_{\bar{i}{i}}$ is just the identity matrix.

In [ ]:
def compute_fock_matrix_from_density(h1, h2, density):
    '''
    Computes Fock matrix from a given density.

    Args:
        h1: 1-body Hamiltonian matrix elements.
        h2: 2-body Hamiltonian matrix elements.
        density: 1-body density matrix.

    Returns:
        f: Fock matrix.
    '''
    f = np.zeros_like(h1)

    # TODO: Your code here

    return f

In [ ]:
# TODO: Your code to show that the Fock matrix for the Hamiltonian is not diagonal.
# Hint: C_ibar,i is the identity.

Diagonalizing the Fock matrix will give you the new transformation coefficients for an improved basis.

Write a function to diagonalize your Fock matrix (Hint: Refer to [`scipy.linalg.eigh`](https://docs.scipy.org/doc/scipy-1.15.0/reference/generated/scipy.linalg.eigh.html) to do this).

In [ ]:
def diagonalize_fock_matrix(f):
    '''
    Diagonalizes the Fock matrix and returns the transformation coefficients C_{ibar, i}.$

    Args:
        f: Fock matrix.

    Returns:
        coeffs: Matrix of expansion coefficients of new basis in terms of starting basis. Specifically,
            coeffs[ibar, :] should be the full eigenvector corresponding to the new state |ibar> = sum_i C_{ibar, i} |i>.
    '''
    coeffs = np.zeros_like(f)

    # TODO: Your code here (hint, look at scipy.linalg.eigh)

    return coeffs

The coefficients you get from this diagonalization can be used to construct a new density matrix. We now want to be sure that this basis is actually better: "better" in the sense that the energy is lower than before. For that we need to evaluate the energy expectation value using the density, not the occupation numbers:

$$
E = \sum_{ij} \rho_{ij} H_{ij} + \frac{1}{2} \sum_{ijkl} \rho_{ij} \rho_{kl} H_{ikjl}.
$$

Write a function to evaluate the energy using the density matrix.

In [ ]:
def evaluate_1and2body_energy_from_density(h1, h2, density):
    '''
    Evaluates energy expectation value of state.

    Args:
        h1: 1-body matrix elements of Hamiltonian.
        h2: 2-body matrix elements of Hamiltonian.
        density: Density matrix of state.

    Returns:
        energy: Energy expectation value of state.
    '''
    energy = 0.0

    # TODO: Your code here

    return energy

Briefly show that the density from the new basis you get from diagonalizing the Fock matrix has a lower energy than the starting density.

In [ ]:
# TODO: Your code here.
# Compare the energy from your first density with your energy from your second density.

To solve the Hartree-Fock equations, we do these steps *iteratively* ($n$ is the iteration number):

1. Construct density matrix $\rho^{(n)}$ from coefficients $C^{(n)}_{\bar{i}i}$.
2. Evaluate energy expectation value $E^{(n)}$.
3. Build Fock operator $F^{(n)}$.
4. Diagonalize Fock operator to get new coefficients $C^{(n + 1)}_{\bar{i}i}$.

This iterative procedure terminates when our final basis is no longer improving in each iteration, so $E^{(n)} \approx E^{(n - 1)}$.

Write a function to put all of these ingredients together to solve Hartree-Fock.

In [ ]:
def solve_hartree_fock_iteratively(h1, h2, occupation_numbers):
    '''
    Solve the Hartree-Fock equations for a given Hamiltonian and occupation numbers.

    Args:
        h1: 1-body matrix elements of Hamiltonian.
        h2: 2-body matrix elements of Hamiltonian.
        occupation_numbers: List of occupation numbers.

    Returns:
        e: HF energy.
        density: HF density matrix.
        coeffs: HF basis coefficients.
        n_iters: Number of iterations required to reach convergence.
    '''
    dim = len(occupation_numbers)
    assert h1.shape == (dim, dim)
    assert h2.shape == (dim, dim, dim, dim)

    e = 0.0
    density = np.zeros((dim, dim))
    coeffs = np.zeros((dim, dim))
    n_iters = 0

    # TODO: Your code here

    return e, density, coeffs, n_iters

In [ ]:
# Code to check HF result for specific values of couplings.

Congratulations, you have solved Hartree-Fock!

Explore how the energy depends on $g / \Delta \epsilon$ and $g_\mathrm{hop} / \Delta \epsilon$. How does the number of iterations depend on $g$, $g_\mathrm{hop}$? Do you have problems with convergence in any cases?